In [71]:
from itertools import product
import time

import pandas as pd
import arxiv

# Define Classes to Be Aggregated

In [94]:
material_classes = ['ceramic','polymer','semiconductor','metal','organometallic']
experiment_classes = ['experiment','simulation']
material_base = ["material"]
all_classes = material_classes + experiment_classes
class_combinations = list(product(material_classes, experiment_classes, material_base))
class_combinations

[('ceramic', 'experiment', 'material'),
 ('ceramic', 'simulation', 'material'),
 ('polymer', 'experiment', 'material'),
 ('polymer', 'simulation', 'material'),
 ('semiconductor', 'experiment', 'material'),
 ('semiconductor', 'simulation', 'material'),
 ('metal', 'experiment', 'material'),
 ('metal', 'simulation', 'material'),
 ('organometallic', 'experiment', 'material'),
 ('organometallic', 'simulation', 'material')]

# Define Helper Functions

In [84]:
def clean_results(r):
    return r.get('summary').replace('\n', ' ')

def get_arxiv_results(query, start, max_results):
    res = arxiv.query(' AND '.join(c), 
                      start = start, 
                      max_results = max_results)
    return [clean_results(r) for r in res]

# Fetch Results from ArXiv

In [85]:
total_results = 100
max_results = 20
query_wait = 0.1

results = []

for c in all_classes:
    print(c)
    n_results = 0
    tmp_array = []
    while n_results < total_results:
        r = get_arxiv_results(' AND '.join(c), 
                          start = n_results, 
                          max_results = max_results)
        if len(r) == 0: break
        n_results = n_results + len(r)
        tmp_array = tmp_array + r
        time.sleep(query_wait)
    results.append({"label":c, "data":tmp_array})

('ceramic', 'experiment', 'material')
('ceramic', 'simulation', 'material')
('polymer', 'experiment', 'material')
('polymer', 'simulation', 'material')
('semiconductor', 'experiment', 'material')
('semiconductor', 'simulation', 'material')
('metal', 'experiment', 'material')
('metal', 'simulation', 'material')
('organometallic', 'experiment', 'material')
('organometallic', 'simulation', 'material')


# Build DataFrame and Output to File

In [122]:
output_file = "./data/arxiv_results.pkl"
df = pd.DataFrame(results)

# One-hot encoded columns
for c in all_classes:
    df[c] = df['label'].apply(lambda x: 1 if c in x else 0)
df.drop('labels', axis=0)

df.to_pickle(output_file)
df

ValueError: labels ['labels'] not contained in axis

# Add One-hot Encoded Columns

In [120]:

df

# Load Previous Results

In [90]:
input_file = "./data/arxiv_results.pkl"
df = pd.read_pickle(input_file)
df

,data,label
0,[ZnO-based scintillation ceramics for applicat...,"(ceramic, experiment, material)"
1,[A three-dimensional lattice of the Josephson ...,"(ceramic, simulation, material)"
2,[Single polymer dynamics offers a powerful app...,"(polymer, experiment, material)"
3,[Polymer chains in colloid-polymer mixtures ca...,"(polymer, simulation, material)"
4,[The utilisation of semiconductor materials as...,"(semiconductor, experiment, material)"
5,[We propose a method to realize diluted magnet...,"(semiconductor, simulation, material)"
6,[Half-metallic ferromagnetism (HMFM) occurs ra...,"(metal, experiment, material)"
7,"[The room temperature structure of aluminum, c...","(metal, simulation, material)"
8,[We present a detailed analysis of the results...,"(organometallic, experiment, material)"
9,[The interaction of trimethyl methylcyclopenta...,"(organometallic, simulation, material)"
